In [2]:
import requests
import random
from pprint import pprint
import sqlite3
from html import unescape
from bs4 import BeautifulSoup 
import re
import time
from fake_useragent import UserAgent
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
import collections 

Я буду доставать отзывы на фильм Ларса фон Триера "Антихрист" с Кинопоиска. Кинопоиск позволяет открыть 200 отзывов сразу, поэтому нам не придется мучиться с доставанием отзывов из разных мест. С помощью Супа распарсю полученный html-файл, достав сначала все хорошие, а затем все плохие отзывы. Положу 35 первых хороших и 35 первых плохих в соответствующие списки. Также сразу сделаю тестовые выборки -- плохих и хороших.

In [38]:
def get_all(url_one):
    session = requests.session()
    ua = UserAgent(verify_ssl=False)
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    good_reviews = soup.findAll('div', {'class': "response good"})
    goodie = []
    for i in good_reviews:
        i_1 = i.find('div', {'class': "brand_words"}).text.replace('\n', '').replace('\xa0', ' ').replace('\r', '')
        goodie.append(i_1)
    goodies = goodie[:35]
    g_test = goodie[35:40]
    soup = BeautifulSoup(page, 'html.parser')
    bad_reviews = soup.findAll('div', {'class': "response bad"})
    badie = []
    for i in bad_reviews:
        i_1 = i.find('div', {'class': "brand_words"}).text.replace('\n', '').replace('\xa0', ' ').replace('\r', '')
        badie.append(i_1)
    badies = badie[:35]
    b_test = badie[35:40]
    return(goodies, badies, g_test, b_test)

С помощью НЛТК и ПайМорфи токенизирую текст и приведу все слова к начальным формам.

In [15]:
def prep(goodies):
    prepr = []
    for i in goodies:
        text_py1 = []
        morph = MorphAnalyzer()
        text_py = [w.lower() for w in word_tokenize(i) if w.isalpha()]
        text_len = len(text_py)
        review = []
        for word in text_py:
            new_word = morph.parse(word)
            new_word1 = new_word[0]
            review.append(new_word1.normal_form)
        review = ' '.join(review)
        prepr.append(review)
    return prepr

Создам множества слов, которые есть только в хороших (а затем только в плохих) отзывах.

In [16]:
def get_unique(first, second):
    true = []
    all_f = ' '.join(first).split()
    all_s = ' '.join(second).split()
    for i in all_f:
        if i not in all_s:
            true.append(i)
    return true

Посчитаю частоту встречаемости каждого из слов в отзывах, уберу те слова, которые встречаются по одному разу. _(Я написала эту функцию, тк она была в задании, но с ней моя accuracy падает, видимо из-за маленькой выборки и красноречия авторов отзывов, поэтому я не буду ее использовать далее в самой программе. Но она рабочая и оставлена здесь, чтобы мне не снизили балл за невыполнение пункта)_

In [17]:
def get_commons(t):
    c = collections.Counter(t)
    commons = [word for word,v in c.items() if v > 1]
    return commons

Определю, хороший отзыв или плохой, посчитав, сколько в нем слов из хорошего и плохого набора. Если слов из хорошего набора больше -- считаем отзыв хорошим, если меньше -- плохим. Если равное количество -- относим его к противоречивым.

In [18]:
def good_or_bad(r, t_g, t_b):
    bads = 0
    goods = 0
    morph = MorphAnalyzer()
    text_py = [w.lower() for w in word_tokenize(r) if w.isalpha()]
    text_len = len(text_py)
    rev = []
    for word in text_py:
        new_word = morph.parse(word)
        new_word1 = new_word[0]
        rev.append(new_word1.normal_form)
    for i in rev:
        if i in t_b:
            bads += 1
        elif i in t_g:
            goods += 1
    if bads > goods:
        ton = 'bad'
    elif bads < goods:
        ton = 'good'
    else:
        ton = 'controversial'
    return ton

Самостоятельно посчитаю accuracy своей функции, разделив количество правильных ответов на общее количество отзывов в тестовой выборке.

In [35]:
def get_acc(test_g, test_b, g_un, b_un):
    acc = 0
    for review in test_g:
        if good_or_bad(review, g_un, b_un) == 'good':
            acc += 1
    for review in test_b:
        if good_or_bad(review, g_un, b_un) == 'bad':
            acc += 1
    acc_tot = acc/(len(test_g)+len(test_b))
    return acc_tot

Теперь запущу все полученные функции.

In [39]:
goodies1, badies1, g_test1, b_test1 = get_all('https://www.kinopoisk.ru/film/408909/reviews/ord/date/status/all/perpage/200/')
goodies2, badies2, g_test2, b_test2 = get_all('https://www.kinopoisk.ru/film/408909/reviews/ord/date/status/all/perpage/200/page/2/')
goodies = goodies1 + goodies2
badies = badies1 + badies2
g_test = g_test1 + g_test2
b_test = b_test1 + b_test2
g_prep = prep(goodies)
b_prep = prep(badies)
g_un = get_unique(g_prep, b_prep)
b_un = get_unique(b_prep, g_prep)

In [40]:
print('accuracy:', get_acc(g_test, b_test, g_un, b_un))

accuracy: 0.6


Чтобы увеличить accuracy, можно:
1. Взять бóльшую выборку отзывов, добавив другие фильмы того же режиссера (они схожей тематики, так что ключевые слова не будут сильно различаться)
2. Можно было бы воспользоваться word2vec, чтобы смотреть не только сами слова, встретившиеся в негативных/позитивных отзывах, но и синонимичные, близкие к ним по векторному представлению
3. Посмотреть не слова, а н-граммы 
4. Найти какую-то черту, отличающую положительные отзывы на этом сайте от отрицательных и смотреть по ней (например, возможно, авторы отрицательных отзывов более эмоциональны в своем гневе, тогда можно будет принимать во внимание знаки препинания).